In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Datasets

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

#### CIFAR-10

In [ ]:
trainset_10 = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader_10 = torch.utils.data.DataLoader(trainset_10, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset_10 = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader_10 = torch.utils.data.DataLoader(testset_10, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Models

MLP-L-H

In [ ]:
class MLP(nn.Module):
  def __init__(self, L, H, fixdim, simplex):
    super().__init__()
    
    self.relu = nn.ReLU()

    layers = [nn.Linear(3072, H)]
    bns = [nn.BatchNorm1d(H)]

    num_classes = 10

    for i in range(L-2):
      layers.append(nn.Linear(H, H))
      bns.append(nn.BatchNorm1d(H))
        
    if fixdim:
      layers.append(nn.Linear(H, num_classes))
      self.fc_out= nn.Linear(num_classes, num_classes)
    else:
      layers.append(nn.Linear(H, H))
      nn.Linear(H, num_classes)
    
    bns.append(nn.BatchNorm1d(H))

    self.fcs = nn.ModuleList(layers)
    self.bns = nn.ModuleList(bns)

    if simplex:
      weight = torch.sqrt(torch.tensor(num_classes/(num_classes-1)))*(torch.eye(num_classes)-(1/num_classes)*torch.ones((num_classes, num_classes)))
      weight /= torch.sqrt((1/num_classes*torch.norm(weight, 'fro')**2))
      if fixdim:
        self.fc_out.weight = nn.Parameter(weight)
      else:
        self.fc_out.weight = nn.Parameter(torch.mm(weight, torch.eye(num_classes, H)))
      self.fc_out.weight.requires_grad_(False)
    
    def forward(self, X):
        for layer, bn in zip(self.fcs, self.bns):
            h = self.relu(bn(layer(h)))
        return self.fc_out(h)

CONV-L-H

In [ ]:
# TODO